In [1]:
#importando as ferramentas
import pandas as pd
import numpy as np

In [2]:
# Define o caminho para o arquivo CSV, nesse caso deve se encontrar na mesma pasta
file_path = 'subscribed_email_audience_export_efc265c2ef.csv'

# Carrega o arquivo CSV
df = pd.read_csv(file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'subscribed_email_audience_export_efc265c2ef.csv'

In [ ]:
#checar tipos de dados e a contagem de valores nulos
print("Informações do DataFrame original:")
df.info()

print("\nPrimeiras 5 linhas do DataFrame original:")
df.head()

# ADICIONE ESTA LINHA PARA VER OS NOMES EXATOS DAS COLUNAS
print(df.columns)

In [ ]:
# Preenche valores ausentes (NaN) com um texto vazio
df[['First Name', 'Last Name', 'Phone Number']] = df[['First Name', 'Last Name', 'Phone Number']].fillna('')

# Padroniza a coluna de Nomes (remove espaços e aplica formato de título)
df['First Name'] = df['First Name'].str.strip().str.title()
df['Last Name'] = df['Last Name'].str.strip().str.title()

# Remove caracteres não numéricos da coluna de Telefone
df['Phone Number'] = df['Phone Number'].str.replace(r'\D', '', regex=True)

# Mostra o resultado da limpeza para confirmar
print("Nomes e Telefones limpos com sucesso.")
df[['First Name', 'Last Name', 'Phone Number']].head()

In [ ]:
# Converte a coluna 'OPTIN_TIME' para o tipo datetime
df['OPTIN_TIME'] = pd.to_datetime(df['OPTIN_TIME'], errors='coerce')

# Imprime uma confirmação e as informações da coluna para verificar
print("Coluna de data 'OPTIN_TIME' convertida com sucesso.")
df[['OPTIN_TIME']].info()

In [ ]:
print("Informações do DataFrame após a limpeza:")
df.info()

print("\nPrimeiras 5 linhas do DataFrame após a limpeza:")
df.head()

In [ ]:
# df.to_csv('audience_cleaned.csv', index=False)
print("="*60)
print("PROCESSO DE LIMPEZA E PREPARAÇÃO DE DADOS CONCLUÍDO COM SUCESSO!")
print("O DataFrame 'df' está pronto para a fase de análise exploratória e visualização.")
print("="*60)

In [ ]:
# Célula de Setup para Análise
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações para melhorar a aparência dos gráficos
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Contagem de usuários por nível de rating, se são novos (rating 2), engajados (rating 3-4) ou VIPs (rating 5)
rating_counts = df['MEMBER_RATING'].value_counts()

# Gráfico de barras para visualizar a distribuição
rating_counts.plot(kind='bar', color='skyblue')
plt.title('Distribuição de Usuários por Member Rating')
plt.xlabel('Rating')
plt.ylabel('Quantidade de Usuários')
plt.xticks(rotation=0) # Mantém os nomes do eixo X na horizontal
plt.show()

In [ ]:
#Analisando tendência e/ou sazonalidade
# Certificando que OPTIN_TIME é do tipo datetime
df['OPTIN_TIME'] = pd.to_datetime(df['OPTIN_TIME'])

# Contando cadastros por mês
cadastros_por_mes = df.set_index('OPTIN_TIME').resample('ME').size()

# Gráfico de linha para visualizar a tendência
cadastros_por_mes.plot(kind='line', marker='o')
plt.title('Tendência de Novos Cadastros ao Longo do Tempo')
plt.xlabel('Data')
plt.ylabel('Quantidade de Cadastros')
plt.show()

In [ ]:
#Análise Geográfica
# Contagem das 10 principais regiões
top_10_regioes = df['REGION'].value_counts().nlargest(10)

# Gráfico de barras horizontais
top_10_regioes.sort_values().plot(kind='barh', color='mediumseagreen')
plt.title('Top 10 Regiões com Maior Quantidade de Usuários')
plt.xlabel('Quantidade de Usuários')
plt.ylabel('Região')
plt.show()

In [ ]:
# --- CÉLULA DE DIAGNÓSTICO PARA O MAPA DE CALOR ---
import folium
from folium.plugins import HeatMap
import pandas as pd

# Etapa de limpeza que já fizemos
df['LATITUDE'] = pd.to_numeric(df['LATITUDE'], errors='coerce')
df['LONGITUDE'] = pd.to_numeric(df['LONGITUDE'], errors='coerce')

# Cria o DataFrame geográfico, removendo linhas com coordenadas inválidas
df_geo = df[['LATITUDE', 'LONGITUDE']].dropna()

# --- INÍCIO DO DIAGNÓSTICO ---
print(f"Número de coordenadas válidas encontradas após a limpeza: {len(df_geo)}")

if len(df_geo) > 0:
    print("Amostra das coordenadas que serão plotadas:")
    print(df_geo.head())
else:
    print("\nAVISO: Nenhum dado de geolocalização válido foi encontrado no arquivo.")
    print("Por isso, o mapa de calor não pôde ser gerado.")
# --- FIM DO DIAGNÓSTICO ---


# Cria o mapa base
mapa = folium.Map(location=[-14.2350, -51.9253], zoom_start=4)

# Apenas tenta adicionar a camada de calor se houver dados
if not df_geo.empty:
    heat_data = [[row['LATITUDE'], row['LONGITUDE']] for index, row in df_geo.iterrows()]
    HeatMap(heat_data).add_to(mapa)

# Exibe o mapa
mapa